In [1]:
import sys
import os
sys.path.insert(0,'../classes')
sys.path.insert(0,'../analysis')
sys.path.insert(0,'/home/zz737/projects/fiar/cog_model/fourinarow/Analysis notebooks/new')
hqfd = '/home/zz737/projects/fiar/cog_model/fourinarow/Analysis notebooks/new/Heuristic quality'
sys.path.insert(0,hqfd)
from fourinarowfunctions import *
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import logging
import numpy as np

import coloredlogs

from arena import Arena
from coach import Coach
from beck.beck_game import BeckGame as Game
from beck.beck_nnet import NNetWrapper as nn
from beck.beck_players import HumanBeckPlayer
from mcts import MCTS
from utils import *
log = logging.getLogger(__name__)

from keras import backend as K

import tournament
# participant_iters = tournament.participant_iters

participants_dir = '/scratch/zz737/fiar/tournaments/tournament_4' # dir for trained networks
results_dir = '/scratch/zz737/fiar/tournaments/results/tournament_4'
moves_dir = '/scratch/zz737/fiar/tournaments/tournament_4/moves/raw/'
splits_dir = '/scratch/zz737/fiar/tournaments/tournament_4/moves/splits/'

import load, plot_fun, anl
from cog_related import cog_value_net as cvn
from importlib import reload
reload(load)
from cog_related import cog_value_net_trainable_mcts as cvntm
from cog_related.cog_value_net_trainable_mcts import NNetWrapper_cog as nnc

{'mcts100_cpuct1': [1, 2, 3, 4, 6, 7, 9, 11, 12, 16, 18, 21, 25, 28, 29, 30, 37, 38, 39, 45, 47], 'mcts100_cpuct2': [1, 2, 3, 4, 6, 8, 12, 14, 15, 16, 21, 22, 24, 26, 27, 30, 32, 35, 39], 'mcts100_cpuct3': [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 16, 17, 18, 20, 21, 25, 26, 28, 32, 34, 35, 36], 'mcts25_cpuct1': [1, 2, 3, 4, 9, 11, 13, 14, 16, 19, 22, 25, 28, 29, 31, 37, 39, 45, 59, 61], 'mcts50_cpuct1': [1, 2, 3, 4, 5, 6, 7, 9, 10, 13, 17, 18, 20, 21, 23, 25, 28, 29, 31, 37, 39, 41, 44, 45, 47, 48, 53], 'mcts80_cpuct1': [1, 2, 3, 4, 5, 7, 8, 9, 10, 13, 16, 19, 20, 21, 23, 24, 28, 34, 35, 36, 40], 'mcts80_cpuct2': [1, 2, 5, 6, 8, 10, 12, 13, 15, 17, 18, 19, 22, 25, 28, 30, 31, 34, 37, 38, 39, 40, 43, 45, 46, 51], 'mcts80_cpuct3': [1, 2, 5, 6, 7, 9, 12, 13, 14, 19, 21, 22, 23, 24, 26, 27, 28, 35, 36, 42, 43, 47, 49, 50, 51, 53, 54, 55, 56, 57]}
189 participant iterations!


In [2]:
g = Game(4,9,4)
nnet = nnc(g)
nnet.load_checkpoint('/scratch/zz737/fiar/models/cog_model_value_mcts/copy1','checkpoint_6.pth.tar')

args = dotdict({
    'display':True,
    'numIters': 1000,
    'numEps': 100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'temp':0,
    'updateThreshold': 0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 100,#100,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 40,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 2,

#     'checkpoint': './temp/',
#     'load_model': True,
    # 'load_folder_file': (f'../models/{model_class_name}',f'{model_instance_name}.pth.tar'),
    # 'load_folder_file': (f'{model_dir}/{model_class_name}',f'{model_instance_name}.pth.tar'),
#     'load_folder_file': (f'{model_dir}',f'{model_instance_name}.pth.tar'),

    'numItersForTrainExamplesHistory': 20,

    # for saving moves
    'is_save_moves': False,
#     'save_moves_folder': f'../moves/{model_class_name}',
#     'save_moves_file': f'{model_instance_name}_temp{temp}.csv', 
#     'overwrite': True, # whether overwrite the existing csv file or add to it

    #
    'numGames':2,
})

nmcts = MCTS(g,nnet,args)

In [4]:
(-1)**True

-1

In [5]:

nnet = nn(g)

In [83]:
checkpoint_folder = '/scratch/zz737/fiar/tournaments/tournament_1/checkpoints_mcts100_cpuct2/'
checkpoint_file = 'checkpoint_39.pth.tar'
nnet.load_checkpoint(checkpoint_folder,checkpoint_file)

In [84]:
board = g.getInitBoard()
board[0:3,0] = 1
board[0:3,1]=-1

In [85]:
board

array([[ 1, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 1, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 1, -1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [87]:
np.set_printoptions(precision=3,suppress=True)
pi,v = nnet.predict(-board)
print(pi)
print(v)

[0.    0.    0.011 0.001 0.001 0.    0.001 0.    0.    0.002 0.    0.
 0.    0.    0.    0.    0.002 0.001 0.    0.    0.    0.    0.002 0.011
 0.    0.    0.001 0.014 0.943 0.002 0.001 0.003 0.001 0.001 0.    0.   ]
[0.953]


In [88]:
np.argmax(pi) % 9

1

In [40]:
ai = lambda x: np.argmax(nmcts.getActionProb(x, temp=0)) #temp=0
human_player = HumanBeckPlayer(g)
human = lambda x: human_player.play(x)
arena = Arena(human, ai, g, display=g.display)
arena.playGame(verbose=args.display, nnet=nnet, is_save_moves=False)

Value of player to go: [0.44861028]
Turn  1 Player  1
   0 1 2 3 4 5 6 7 8 
-----------------------
0 |- - - - - - - - - |
1 |- - - - - - - - - |
2 |- - - - - - - - - |
3 |- - - - - - - - - |
-----------------------
[ 0 0] [ 0 1] [ 0 2] [ 0 3] [ 0 4] [ 0 5] [ 0 6] [ 0 7] [ 0 8] [ 1 0] [ 1 1] [ 1 2] [ 1 3] [ 1 4] [ 1 5] [ 1 6] [ 1 7] [ 1 8] [ 2 0] [ 2 1] [ 2 2] [ 2 3] [ 2 4] [ 2 5] [ 2 6] [ 2 7] [ 2 8] [ 3 0] [ 3 1] [ 3 2] [ 3 3] [ 3 4] [ 3 5] [ 3 6] [ 3 7] [ 3 8] exit
Invalid move


KeyboardInterrupt: Interrupted by user

In [15]:
nnet.nnet.model.get_weights()

[array([[[[ 0.05485117, -0.12358797,  0.12437216, ..., -0.14390627,
            0.10108473,  0.14895956]],
 
         [[-0.03292236, -0.04945462,  0.01874612, ..., -0.01183348,
           -0.06147672, -0.01145913]],
 
         [[ 0.01904835, -0.11642803, -0.11939302, ...,  0.15198669,
            0.09225779, -0.10802178]]],
 
 
        [[[ 0.12009411, -0.08946774,  0.01234221, ..., -0.03620895,
            0.14028847, -0.02482955]],
 
         [[ 0.15355828, -0.00409129,  0.06318402, ..., -0.0876192 ,
           -0.00372958,  0.12756583]],
 
         [[ 0.05389964, -0.09288926,  0.04811531, ...,  0.06945518,
           -0.13018008, -0.07878888]]],
 
 
        [[[-0.04926426,  0.02629192, -0.1381113 , ..., -0.12721612,
            0.09294331,  0.0058242 ]],
 
         [[ 0.010053  , -0.0153363 , -0.02288041, ...,  0.02030519,
            0.07006372, -0.0308173 ]],
 
         [[-0.15849125,  0.12446816,  0.16743827, ...,  0.11848098,
            0.0454274 ,  0.1472098 ]]]], dtype=float32

In [3]:

board = g.getInitBoard()

In [24]:
nnetwp = cvntm.NNetWrapper_cog(g)

In [26]:
N_feats = 5

args = dotdict({
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 10,
    'batch_size': 64,
    'cuda': False,
    'num_channels': 512,
    'N_feats':N_feats
})

In [40]:
reload(cvntm)
nnetwp = cvntm.NNetWrapper_cog(g)

In [41]:
nnetwp.predict(board)

(array([0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
        0.02777778], dtype=float32),
 array([0.], dtype=float32))

In [42]:
pi = np.zeros(36)

In [44]:
pi[0] = 1

In [47]:
examples = [(board, pi, 1)]

In [68]:
nnetwp.nnet.model.layers[-1].get_weights()

[array([[ 0.30330998],
        [ 0.6466014 ],
        [-0.31532502],
        [-0.57625145],
        [-0.02030754],
        [-0.01113045],
        [-0.11489326],
        [-0.4275066 ],
        [-0.6517888 ],
        [-0.7361808 ]], dtype=float32),
 array([0.00999671], dtype=float32)]

In [48]:
nnetwp.train(examples)

Epoch 1/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5835 - pi_loss: 3.5835 - v_loss: 1.0000
Epoch 2/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5796 - pi_loss: 3.5816 - v_loss: 0.9980
Epoch 3/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5756 - pi_loss: 3.5796 - v_loss: 0.9960
Epoch 4/10
1/1 [==============================] - 0s 977us/step - loss: 4.5717 - pi_loss: 3.5777 - v_loss: 0.9940
Epoch 5/10
1/1 [==============================] - 0s 983us/step - loss: 4.5678 - pi_loss: 3.5757 - v_loss: 0.9920
Epoch 6/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5638 - pi_loss: 3.5738 - v_loss: 0.9900
Epoch 7/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5599 - pi_loss: 3.5719 - v_loss: 0.9880
Epoch 8/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5560 - pi_loss: 3.5699 - v_loss: 0.9861
Epoch 9/10
1/1 [==============================] - 0s 1ms/step - loss: 4.5520 - pi_loss: 3.5680 - v_l

In [11]:
cvn = CogValueNet(g,args)

In [18]:
board = board[None,:,:]
feats = np.zeros((1,10))
cvn.model.predict([board,feats])

[array([[0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778, 0.02777778, 0.02777778, 0.02777778, 0.02777778,
         0.02777778]], dtype=float32),
 array([[0.]], dtype=float32)]

In [31]:
class B(A):
    def __init(self):
        self.b=2
    

In [32]:
b = B()

In [33]:
b.dosth()

hi


In [94]:
checkpoint = '/scratch/zz737/fiar/models/cog_model_value_mcts/copy1/'
checkpoint_folder = '/scratch/zz737/fiar/tournaments/tournament_1/checkpoints_mcts100_cpuct2/'
checkpoint_file = 'checkpoint_39.pth.tar'

args = dotdict({
    'numIters': 2,#1000,
    'numEps': 2,#100,              # Number of complete self-play games to simulate during a new iteration.
    'tempThreshold': 15,        #
    'updateThreshold': 0,#0.6,     # During arena playoff, new neural net will be accepted if threshold or more of games are won.
    'maxlenOfQueue': 200000,    # Number of game examples to train the neural networks.
    'numMCTSSims': 10,#100,          # Number of games moves for MCTS to simulate.
    'arenaCompare': 1,#60,         # Number of games to play during arena play to determine if new net will be accepted.
    'cpuct': 2,

    # 'checkpoint': './scratch/zz737/fiar/tournaments/tournament_4/checkpoints_mcts100_cpuct2_id10/',
    'checkpoint': checkpoint_folder,#checkpoint,
    'load_model': False,#False,
    'load_folder_file': (checkpoint_folder,checkpoint_file),
    # 'load_folder_file': (checkpoint,f'checkpoint_{loaded_iter}.pth.tar'),
    'numItersForTrainExamplesHistory': 20,

    'loaded_iter': -1,#loaded_iter, #[SZ] by default -1, if loading a checkpoint, use that iter 

})

In [95]:
# C = Coach(g, nnetwp, args)
C = Coach(g, nnet, args)

In [96]:
C.loadTrainExamples()

In [97]:
C.trainExamplesHistory[-1][-1]

(array([[ 0, -1,  1,  1,  0,  0,  0,  0,  0],
        [ 0,  0,  1,  0, -1,  0,  0,  0,  0],
        [ 0,  1,  1,  1,  0,  0,  0,  0,  0],
        [-1,  0, -1, -1,  0, -1,  0,  0,  0]]),
 array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.262, 0.   , 0.   , 0.   , 0.672, 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.067, 0.   , 0.   , 0.   , 0.   ]),
 1)

In [133]:
test_board_str = '''[[-1  0  1  1 -1  0  0  0  0] 
 [ 0  0  0 -1  1  1  0  0  0] 
 [ 0  0 -1  1 -1 -1  0  0  0] 
 [ 0  1  1 -1 -1 -1  1  0  0]]'''

test_board_str= test_board_str.replace('[','').replace(']','').replace('\n',' ').split(' ')
test_board = np.array([int(i) for i in test_board_str if i!='']).reshape(4,9)

In [134]:
test_board

array([[-1,  0,  1,  1, -1,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  1,  1,  0,  0,  0],
       [ 0,  0, -1,  1, -1, -1,  0,  0,  0],
       [ 0,  1,  1, -1, -1, -1,  1,  0,  0]])

In [135]:
nnet.predict(test_board)

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.], dtype=float32),
 array([0.008], dtype=float32))

In [140]:
C.trainExamplesHistory[-1][44]

(array([[ 0,  0,  0,  0, -1,  1,  0,  1,  0],
        [ 0,  0,  0, -1,  1, -1,  0,  0,  0],
        [ 0,  0, -1,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  1, -1, -1,  0,  1,  0]]),
 [0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.0,
  0.0,
  0.02336448598130841,
  0.0,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.0,
  0.0,
  0.0,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.0,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.004672897196261682,
  0.8691588785046729,
  0.004672897196261682,
  0.0,
  0.0,
  0.0,
  0.004672897196261682,
  0.0,
  0.004672897196261682],
 -1)

In [144]:
tot = 0
win = 0
inv_dist_to_center = cvn.get_inv_dist_to_center(g)
for ii,ex in enumerate(C.trainExamplesHistory[-1]):
    b,pi,val = ex
    feat_vals, feat_names = cvn.get_all_feat(-b,inv_dist_to_center)
    if feat_vals[3] > 0:
        tot+=1
        if val == -1:
            win += 1
#         if val!=1:
#             print(ii)
#             print(val)
#             print(b)
#             print('/n')

In [145]:
print(tot,win)

1616 500


In [143]:
print(tot,win)

308 284


In [ ]:
for b in C.trainExamplesHistory[-1]:
    cvn.get_all_feat()

In [98]:
C.learn()

Self Play: 100%|██████████| 2/2 [00:14<00:00,  7.02s/it]


Checkpoint Directory exists! 
Epoch 1/10
3/3 [==============================] - 1s 330ms/step - loss: 5.5434 - pi_loss: 4.0276 - v_loss: 1.5157
Epoch 2/10
3/3 [==============================] - 1s 328ms/step - loss: 4.7941 - pi_loss: 3.2805 - v_loss: 1.5136
Epoch 3/10
3/3 [==============================] - 1s 320ms/step - loss: 4.5242 - pi_loss: 3.0133 - v_loss: 1.5109
Epoch 4/10
3/3 [==============================] - 1s 320ms/step - loss: 4.3335 - pi_loss: 2.8255 - v_loss: 1.5080
Epoch 5/10
3/3 [==============================] - 1s 320ms/step - loss: 4.2277 - pi_loss: 2.7230 - v_loss: 1.5047
Epoch 6/10
3/3 [==============================] - 1s 320ms/step - loss: 4.1662 - pi_loss: 2.6646 - v_loss: 1.5016
Epoch 7/10
3/3 [==============================] - 1s 319ms/step - loss: 4.0296 - pi_loss: 2.5314 - v_loss: 1.4983
Epoch 8/10
3/3 [==============================] - 1s 318ms/step - loss: 4.0113 - pi_loss: 2.5165 - v_loss: 1.4948
Epoch 9/10
3/3 [==============================] - 1s 323ms

Arena.playGames (1): 0it [00:00, ?it/s]
Arena.playGames (2): 0it [00:00, ?it/s]

Checkpoint Directory exists! 



Self Play: 100%|██████████| 2/2 [00:11<00:00,  5.74s/it]


Checkpoint Directory exists! 
Epoch 1/10
4/4 [==============================] - 2s 559ms/step - loss: 5.7061 - pi_loss: 4.1268 - v_loss: 1.5792
Epoch 2/10
4/4 [==============================] - 2s 558ms/step - loss: 4.9284 - pi_loss: 3.3548 - v_loss: 1.5736
Epoch 3/10
4/4 [==============================] - 2s 586ms/step - loss: 4.5373 - pi_loss: 2.9727 - v_loss: 1.5646
Epoch 4/10
4/4 [==============================] - 2s 585ms/step - loss: 4.3332 - pi_loss: 2.7776 - v_loss: 1.5556
Epoch 5/10
4/4 [==============================] - 2s 584ms/step - loss: 4.1519 - pi_loss: 2.6051 - v_loss: 1.5468
Epoch 6/10
4/4 [==============================] - 2s 588ms/step - loss: 3.9506 - pi_loss: 2.4123 - v_loss: 1.5382
Epoch 7/10
4/4 [==============================] - 2s 584ms/step - loss: 3.8561 - pi_loss: 2.3275 - v_loss: 1.5287
Epoch 8/10
4/4 [==============================] - 2s 588ms/step - loss: 3.7699 - pi_loss: 2.2517 - v_loss: 1.5181
Epoch 9/10
4/4 [==============================] - 2s 590ms

Arena.playGames (1): 0it [00:00, ?it/s]
Arena.playGames (2): 0it [00:00, ?it/s]

Checkpoint Directory exists! 


In [100]:
board[]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [107]:
C.nnet.nnet.model.get_weights()[-2]

array([[ 0.30330998],
       [ 0.6466014 ],
       [-0.31532502],
       [-0.57625145],
       [-0.02030754],
       [-0.01113045],
       [-0.11489326],
       [-0.4275066 ],
       [-0.6517888 ],
       [-0.7361808 ]], dtype=float32)

In [99]:
C.nnet.predict(board)

(array([0.02832301, 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 , 0.0277622 ,
        0.0277622 ], dtype=float32),
 array([0.00999638], dtype=float32))

In [79]:
C.mcts.Qsa

{(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00